<a href="https://colab.research.google.com/github/erikluu/QuantifyingHappiness/blob/main/Data_Collection_and_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import pandas as pd
import requests
from functools import reduce

# Cleaning Kaggle Data

In [185]:
k_2015 = pd.read_csv("2015.csv")
k_2016 = pd.read_csv("2016.csv")
k_2017 = pd.read_csv("2017.csv")
k_2018 = pd.read_csv("2018.csv")
k_2019 = pd.read_csv("2019.csv")

# common_columns = ['Overall rank', 'Country or region', 'Score', 'GDP per capita',
#        'Social support', 'Healthy life expectancy',
#        'Freedom to make life choices', 'Generosity',
#        'Perceptions of corruption']    

k_2015.isnull().values.any(), k_2016.isnull().values.any(), k_2017.isnull().values.any(), k_2018.isnull().values.any(), k_2019.isnull().values.any() 

(False, False, False, True, False)

In [186]:
# clean 2019
k_2019 = k_2019.rename(columns={"Country or region": "Country"})
k_2019["Year"] = 2019

# clean 2018 - same columns as 2019 but has NaN values in percetptions of corruption
# k_2018['Perceptions of corruption'].isnull().values.any()
k_2018 = k_2018.rename(columns={"Country or region": "Country"})
k_2018 = k_2018.fillna(k_2018["Perceptions of corruption"].mean())
k_2018["Year"] = 2018

# clean 2017 - match columns to 2019
k_2017 = k_2017.drop(columns=["Whisker.high", "Whisker.low", "Dystopia.Residual"]) 
k_2017.columns = ['Country', 'Overall rank', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']
k_2017["Year"] = 2017
       
# clean 2016 - match columns to 2019
country_region = dict(zip(k_2016["Country"], k_2016["Region"]))
k_2016 = k_2016.drop(columns=["Lower Confidence Interval", "Upper Confidence Interval", "Dystopia Residual", "Region"])
k_2016.columns = ['Country', 'Overall rank', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices',
       'Perceptions of corruption', 'Generosity'] 
k_2016["Year"] = 2016

# clean 2015 - match columns to 2019
k_2015 = k_2015.drop(columns=["Standard Error", "Dystopia Residual", "Region"])
k_2015.columns = ['Country', 'Overall rank', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices',
       'Perceptions of corruption', 'Generosity']
k_2015["Year"] = 2015       

In [198]:
df_merged = k_2019.append([k_2018, k_2017, k_2016, k_2015]).reset_index()
df_merged = df_merged.drop("index", axis=1)
df_merged["Region"] = df_merged["Country"].map(country_region)
df_merged

,Overall rank,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Region
0,1,Finland,7.769,1.34000,1.58700,0.98600,0.59600,0.15300,0.39300,2019,Western Europe
1,2,Denmark,7.600,1.38300,1.57300,0.99600,0.59200,0.25200,0.41000,2019,Western Europe
2,3,Norway,7.554,1.48800,1.58200,1.02800,0.60300,0.27100,0.34100,2019,Western Europe
3,4,Iceland,7.494,1.38000,1.62400,1.02600,0.59100,0.35400,0.11800,2019,Western Europe
4,5,Netherlands,7.488,1.39600,1.52200,0.99900,0.55700,0.32200,0.29800,2019,Western Europe
...,...,...,...,...,...,...,...,...,...,...,...
777,154,Rwanda,3.465,0.22208,0.77370,0.42864,0.59201,0.22628,0.55191,2015,Sub-Saharan Africa
778,155,Benin,3.340,0.28665,0.35386,0.31910,0.48450,0.18260,0.08010,2015,Sub-Saharan Africa
779,156,Syria,3.006,0.66320,0.47489,0.72193,0.15684,0.47179,0.18906,2015,Middle East and Northern Africa
780,157,Burundi,2.905,0.01530,0.41587,0.22396,0.11850,0.19727,0.10062,2015,Sub-Saharan Africa


# Adding Geographic (lat, lng) data from https://world-happiness-database.herokuapp.com

In [199]:
response = requests.get("https://world-happiness-database.herokuapp.com/api/geographic")
r = response.json()

countries = []
lat = []
lng = []
for c in r:
  countries.append(c["country"])
  lat.append(c["lat"])
  lng.append(c["lon"])

dict_lat = dict(zip(countries, lat))
dict_lng = dict(zip(countries, lng))

df_merged["lat"] = df_merged["Country"].map(dict_lat)
df_merged["lng"] = df_merged["Country"].map(dict_lng)

In [200]:
df_merged

,Overall rank,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Region,lat,lng
0,1,Finland,7.769,1.34000,1.58700,0.98600,0.59600,0.15300,0.39300,2019,Western Europe,61.924110,25.748151
1,2,Denmark,7.600,1.38300,1.57300,0.99600,0.59200,0.25200,0.41000,2019,Western Europe,56.263920,9.501785
2,3,Norway,7.554,1.48800,1.58200,1.02800,0.60300,0.27100,0.34100,2019,Western Europe,60.472024,8.468946
3,4,Iceland,7.494,1.38000,1.62400,1.02600,0.59100,0.35400,0.11800,2019,Western Europe,64.963051,-19.020835
4,5,Netherlands,7.488,1.39600,1.52200,0.99900,0.55700,0.32200,0.29800,2019,Western Europe,52.132633,5.291266
...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,154,Rwanda,3.465,0.22208,0.77370,0.42864,0.59201,0.22628,0.55191,2015,Sub-Saharan Africa,-1.940278,29.873888
778,155,Benin,3.340,0.28665,0.35386,0.31910,0.48450,0.18260,0.08010,2015,Sub-Saharan Africa,9.307690,2.315834
779,156,Syria,3.006,0.66320,0.47489,0.72193,0.15684,0.47179,0.18906,2015,Middle East and Northern Africa,34.802075,38.996815
780,157,Burundi,2.905,0.01530,0.41587,0.22396,0.11850,0.19727,0.10062,2015,Sub-Saharan Africa,-3.373056,29.918886
